In [1]:
from bs4 import BeautifulSoup
import requests
import time
import re
import glob
import os
import codecs
import mysql.connector
from pandas import DataFrame
import pandas as pd

In [4]:
headers = {'User-Agent': 'Mozilla/5.0'}

### Scrapping Ebay

In [6]:
ebay_url = 'https://www.ebay.com/sch/i.html?'
keyword  = "_nkw=playstation+4+slim&"
pagelmt  = "_ipg=100&"
buyitnow = "LH_BIN=1&"
pgn      = "_pgn="

COMMENT : For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)

COMMENT: Checksum is in the link for only the sponsored hrefs therefore using this as the regex. 
Did not use *S.*P.*O.*N.*S.*O.*R.*E.*D as it was tagging unsponsered as sponsored because of the title text having the same letters in the order ex: -S-ony -P-laystati-ON con-SO-le b-R-and n-E-w -D-evice

In [6]:
sponsored_file     = open('sponsored.txt', 'w')
non_sponsored_file = open('non-sponsored.txt', 'w')

for i in range(1,11):
    ebay_url_final  = ebay_url+keyword+pagelmt+buyitnow+pgn+str(i)
    print(ebay_url_final)
    ebay_page = requests.get(ebay_url_final, headers = headers)

    soup   = BeautifulSoup(ebay_page.text, 'html.parser')
    a_href = soup.find_all('a', class_ = "s-item__link")
    sponsor_count = 0

    for a in a_href:
        sponsor  = re.findall(r'.*checksum*', a['href'])
        item_url = re.sub("(.*?)\?.*", "\\1", a['href'])
        
        if sponsor != []:
            sponsor_count = sponsor_count+1 
            sponsored_file.write("%s\n" % item_url)
        else:
            non_sponsored_file.write("%s\n" % item_url)
        time.sleep(1)

#checking if nonsponsor count is 100 in each of the pages
    print("{}: sponsor:{}/{} nonsponsor:{}/{}".format(i,sponsor_count, len(a_href),  len(a_href)-sponsor_count, len(a_href)))

sponsored_file.close()
non_sponsored_file.close()        
            

https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&LH_BIN=1&_pgn=1
1: sponsor:12/112 nonsponsor:100/112
https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&LH_BIN=1&_pgn=2
2: sponsor:10/110 nonsponsor:100/110
https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&LH_BIN=1&_pgn=3
3: sponsor:13/113 nonsponsor:100/113
https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&LH_BIN=1&_pgn=4
4: sponsor:13/113 nonsponsor:100/113
https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&LH_BIN=1&_pgn=5
5: sponsor:13/113 nonsponsor:100/113
https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&LH_BIN=1&_pgn=6
6: sponsor:13/113 nonsponsor:100/113
https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&LH_BIN=1&_pgn=7
7: sponsor:13/113 nonsponsor:100/113
https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&LH_BIN=1&_pgn=8
8: sponsor:13/113 nonsponsor:100/113
https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&

### Download the pages and scrape elements

COMMENT : The URLs saved in text files are dowloaded into the folder using three functions :
    dump_all_urls_to_file : takes url and creates the folder to which the file needs to be downloaded
    create_path : creates the path to download
    download_html : downloads the html

In [8]:
def download_html(url,file_path):
    headers = {'User-Agent': 'Mozilla/5.0'}
    attempt = 1
    allowed_attempts = 5
    
    while attempt <= allowed_attempts:                   #Exception handling: try attempting connecting 5 times before
        try:
            page = requests.get(url, headers = headers)
            break
        except Exception as err:
            
            print("attempt {}/{}: Error while fetching {}\n {}\n".format(attempt, allowed_attempts, url, err))
            time.sleep(5)
            attempt = attempt + 1

    soup = BeautifulSoup(page.text, 'html.parser')
    file = codecs.open(file_path, 'w',"utf-8")
    file.write(page.text)
    file.close()
    
def create_path(url, folder_name):
    
    file_id   = url.split("/")[-1]
    file_path = "{}/{}.html".format(folder_name,file_id)
    if os.path.exists(file_path):
        print("exists")
        return
    download_html(url,file_path)
    #time.sleep(1)

def dump_all_urls_to_file(meta_data_file):
    folder_name = meta_data_file.split(".")[0]
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    for counter, l in enumerate(open(meta_data_file)):
        l = l.split("\n")[0]
        create_path(l, folder_name)
        print(counter,l)                                      
        
dump_all_urls_to_file("sponsored.txt")                        #Calling the nested function to download links into folder
dump_all_urls_to_file("non-sponsored.txt")                    #Calling the nested function to download links into folder
        

exists
0 https://www.ebay.com/itm/Sony-PlayStation-4-PS4-Slim-1tb-Jet-Black-Console-w-accessories-SHIPS-FAST/323649342133
exists
1 https://www.ebay.com/itm/New-Sony-PlayStation-4-PS4-Slim-1TB-Console-3-Game-Bundle-FREE-2-3-DAY-DELIVERY/184075784607
exists
2 https://www.ebay.com/itm/Sony-PlayStation-4-PS4-Slim-1TB-Jet-Black-Console-Power-Cord-HDMI-Cable-Only/273845805342
exists
3 https://www.ebay.com/itm/Sony-PlayStation-4-PS4-Slim-1TB-Console-3-Game-Bundle-Same-Day-Shipping/153756945584
exists
4 https://www.ebay.com/itm/Brand-New-Sony-Playstation-4-PS4-Slim-1TB-Console-Jet-Black-Plus-3-Games/223896948189
exists
5 https://www.ebay.com/itm/Sony-Playstation-4-Slim-Black-500GB-CUH-2015A-Console-Only-US-STOCK/193334876682
exists
6 https://www.ebay.com/itm/Brand-New-Sony-PlayStation-4-Slim-1TB-Only-on-PlayStation-Console-Bundle/113988512101
exists
7 https://www.ebay.com/itm/Sony-PlayStation-4-Slim-1TB-Console-3-Game-Bundle-FREE-DELIVERY-US-WARRANTY/254497606912
exists
8 https://www.ebay.com/

exists
642 https://www.ebay.com/itm/Playstation-4-Slim-1TB-Jet-Black-Gaming-Console-W-Death-Stranding/254421855345
exists
643 https://www.ebay.com/itm/Sony-PlayStation-4-Slim-1TB-Console-Jet-Black/372915527385
exists
644 https://www.ebay.com/itm/Sony-PlayStation-4-Slim-1TB-Black-Console-With-7-games/372923395208
exists
645 https://www.ebay.com/itm/Sony-PlayStation-4-Slim-1TB-Jet-Black-Game-Console-CUH-2215B-3003348/223658154561
exists
646 https://www.ebay.com/itm/Sony-PlayStation-4-Slim-1TB-Console-CUH-2215B-0792/372840428443
exists
647 https://www.ebay.com/itm/Sony-PlayStation-4-Slim-1TB-Black-Gaming-Console/113751776516
exists
648 https://www.ebay.com/itm/New-SONY-Playstation-PS4-Slim-HDR-1-TB-Bundle-God-of-War-Horizon-The-Last-of-US/312867173416
exists
649 https://www.ebay.com/itm/New-Sony-PlayStation-4-PS4-Slim-1TB-Console-3-Game-Bundle-FREE-2-3-DAY-DELIVERY/264605082493
exists
650 https://www.ebay.com/itm/Brand-New-Sony-PlayStation-4-PS4-Slim-1TB-Console-3-Game-Bundle-FREE-DELIVER

seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition

In [9]:
################################# Dictonary with mapping for each of the attributes of interest ############################

field_parser= {
    "seller_name":{                                                          
        "tag": "span",                                                       
        "property_type":"class_",
        "property_value":"mbg-nw"                                               #identifier for seller name
    },
    "seller_score":{
        "tag": "span",
        "property_type":"class_",
        "property_value":"mbg-l"                                                #identifier for seller score
        },
    "item_price":{
        "tag": "span",
        "property_type":"class_",                                                # not using id = prcIsum as in few cases id = price
        "property_value":"notranslate"                                           #identifier for seller price 
        },
    "items_sold":{
        "tag": "a",
        "property_type":"class_",
        "property_value":"vi-txt-underline"                                      #identifier for items sold
        },
    "best_offer":{
        "tag": "div",
        "property_type":"class_",
        "property_value":"val"                                                   #identifier for best offer
    },
    "title": {
        "tag": "h1",
        "property_type":"class_",
        "property_value":"it-ttl"                                                #identifier for title 
    },
    "return": {
        "tag": "div",
        "property_type":"id",
        "property_value":"why2buy",
        "regex_condition": "(.*Returns accepted.*)"                             #identifer and regec for Returns
    },
    "shipping_price": {
        "tag": "span",
        "property_type":"id",
        "property_value":"fshippingCost",                                       #identifier for shipping cost
    },
    "condition":{
        "tag": "div",
        "property_type":"id",
        "property_value":"vi-itm-cond",                                         #identifier for condition 
        
    }
}

COMMENT || FUNCTION: get_features_from_soup : takes soup and dictionary mapping as input and returns dictionary with mapping for seller score, seller name, return allowed etc 

In [10]:
def get_features_from_soup(soup, feature_map):
    output = {}
    for feature in feature_map:
        tag            = feature_map[feature]["tag"]
        prop           = feature_map[feature]["property_type"]
        prop_val       = feature_map[feature]["property_value"]
        
        if "regex_condition" in  feature_map[feature]:                  #in case we have a regex in feature dict (ex:returnss)
            regex_condition = feature_map[feature]["regex_condition"]
        else:
            regex_condition = None
            
         
        if prop   == "class_":                                          #creating a block for variable tags
            block = soup.find_all(tag,class_ = prop_val)
        elif prop == "id":
            block = soup.find_all(tag,id = prop_val)
        else:
            raise Exception
        

        if len(block)>0:                                                #in case of regex (ex:returns)
            if regex_condition:
                value = re.match(regex_condition,str(block[0]), re.DOTALL) is not None
            else:
                value = block[0].text

        else:
            value = None
        output[feature]=value
    return output


COMMENT || Function get_data_from_folder taking input as to which folder to taken in as input and returns a list of dictionary (python object) with mapping of all items in 10 pages for the keyword with mapping for seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition

In [11]:
def get_data_from_folder(folder):
    files = os.listdir(folder)
    output = []
    for file_name in files:
        f = open("./{}/{}".format(folder,file_name), "r", encoding="utf-8").read()  #creating the path 
        soup = BeautifulSoup(f)                               
        features = get_features_from_soup(soup, field_parser)
        features["item_id"] = file_name.split(".")[0]                               #add item id
        features["sponsored"] = folder.split("./")[-1]                              #add if the item is sponsered or non-sponored
        output.append(features)
    return output


### Connect to mySQL

The code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Database ebay is created, info is stored in "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored. Missing information is saved as NULL. 

In [2]:
###########################################################  conneting to MYSQL 
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True)
mycursor = mydb.cursor()

In [ ]:
mycursor.execute("""DROP DATABASE IF EXISTS eBay""")
mycursor.execute("""CREATE DATABASE eBay""")
mycursor.execute("""USE eBay""")
mydb.commit()

In [23]:
mycursor.execute
# items sold, best offer available, title, returns allowed, shipping price, condition
mycursor.execute("DROP TABLE IF EXISTS eBay_items")

mycursor.execute("""CREATE TABLE eBay_items
(
Item_ID VARCHAR(100) PRIMARY KEY ,seller_name VARCHAR(100),seller_score INT,item_price VARCHAR(100),
items_sold INT, best_offer_available VARCHAR(100), title VARCHAR(100),
returns_allowed VARCHAR(10),shipping_price INT,condition_ VARCHAR(50),sponsored VARCHAR(20)
)
""")
dict_into_sql = """INSERT INTO eBay_items
                (Item_ID,seller_name, seller_score, item_price, 
                items_sold, best_offer_available, title, 
                returns_allowed, shipping_price,condition_,sponsored) 
                VALUES (%s,%s,%s,%s,
                        %s,%s,%s,
                        %s,%s,%s,%s)"""
mydb.commit()

In [24]:
output_spons           = get_data_from_folder("./sponsored")      #get dict with attribute and value mapping for sponsored folder
output_non_spons       = get_data_from_folder("./non-sponsored")  #get dict with attribute and value mapping for non-sponsored folder
output                 = output_spons + output_non_spons          #get dict with attribute and value mapping for all
id_set=set()

for element in output:
    item_id            = element["item_id"]
    if item_id in id_set:
        continue
    else:
        id_set.add(item_id)
    seller_name        = element["seller_name"]
    seller_score       = int(re.findall('\d+', str(element['seller_score']))[0])
    
    #condition for item price to extract and convert to int 
    item_price         = re.findall('\d+', str(element['item_price'])) if element['item_price']!= None else None
    item_price_int     = int(item_price[0])*100+int(item_price[1]) if item_price != None else None
    
    items_sold         = int(re.findall('\d+', str(element['items_sold']))[0]) if element['items_sold']!= None else None
    best_offer         = element["best_offer"] if element['best_offer']!= None else None
    title              = element["title"].split("\xa0")[-1]
    return_type        = element["return"] if element['return']!= None else None
    condition_         = element["condition"]
    sponsored          = element["sponsored"]

##################################################### items sold with conditions 
    
    if (element['items_sold']) == None:                                   #if we do not have the information
        items_sold = None
    elif(element['items_sold'])!= None:
        items_sold_temp = re.findall('\d+', str(element['items_sold']))   #getting digits
        if (len(items_sold_temp) == 2):                                   #if items_sold_temp = 1,200 
            items_sold  = (items_sold_temp[0])+(items_sold_temp[1])         
        else:
            items_sold  = int(items_sold_temp[0])                          
                                                    
#################################################### shipping price with contitions 
    if element['shipping_price']== None:
        shipping_price  = None
    elif("FREE" in element['shipping_price']):
        shipping_price  = 0
    else:
        shp_price_temp  = re.findall('\d+', str(element['shipping_price']))
        shipping_price  = int(shp_price_temp[0])*100+int(shp_price_temp[1])
        
    value = (item_id,seller_name, seller_score, item_price_int,
             items_sold, best_offer,title, 
             return_type, shipping_price,condition_,sponsored)
    
   
    mycursor.execute(dict_into_sql, value)
    mydb.commit()

mydb.close()

In [25]:
print(len(output_non_spons))  #<1000 due to duplicates
print(len(output_spons))     # no duplicates found and is seen here

1040
132


### Write to SQL

In [3]:


mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True)
mycursor = mydb.cursor()

mycursor.execute("""USE eBay""")


####################################### Statistics #########################################
mycursor.execute("""SELECT 

sponsored,condition_,
round(avg(seller_score),2),max(seller_score),min(seller_score),round(stddev(seller_score),2),
round(avg(item_price),2),max(item_price),min(item_price),round(stddev(item_price),2),
round(avg(best_offer_available),2),max(best_offer_available),min(best_offer_available),round(stddev(best_offer_available),2),
round(avg(items_sold),2),max(items_sold),min(items_sold),round(stddev(items_sold),2),
round(avg(shipping_price),2),max(shipping_price),min(shipping_price),round(stddev(shipping_price),2),
count(seller_name),sum(returns_allowed)

FROM ebay_items

group by sponsored,condition_
order by condition_,sponsored""")

myresult = mycursor.fetchall()

Data_with_Condition= pd.DataFrame(myresult, columns = [
    "Sponsored","Condition",
    "Average Seller Score","Max Seller score","Min Seller score","Std dev Seller score",
    "Average Item price","Max Item price","Min Item price","Std dev Item price",
    "Average Best offer","Max Best offer","Min Best offer","Std dev Best offer",
    "Average Items sold","Max Items sold","Min Items sold","Std dev Items sold",
    "Average shipping price","Max shipping price","Min shipping price","Std dev shipping price",
    "Number of seller","# Returns allowed"
]) 
Data_with_Condition
mydb.commit()

In [4]:
Data_with_Condition

,Sponsored,Condition,Average Seller Score,Max Seller score,Min Seller score,Std dev Seller score,Average Item price,Max Item price,Min Item price,Std dev Item price,...,Average Items sold,Max Items sold,Min Items sold,Std dev Items sold,Average shipping price,Max shipping price,Min shipping price,Std dev shipping price,Number of seller,# Returns allowed
0,non-sponsored,Manufacturer refurbished,453213.20,2266011,0,906398.90,21359.80,25000,19000,2148.94,...,5.00,5,5,0.00,0.00,0,0,0.00,5,2.0
1,sponsored,Manufacturer refurbished,295.00,534,56,239.00,26949.50,33900,19999,6950.50,...,3.00,3,3,0.00,0.00,0,0,0.00,2,1.0
2,non-sponsored,New,15415.99,916889,0,66781.91,31124.71,69999,14300,8602.36,...,74.92,1896,1,282.87,392.22,4854,0,926.83,373,186.0
3,sponsored,New,14097.14,568398,0,78693.01,33593.16,59999,22100,9880.01,...,29.24,317,1,68.14,456.60,4999,0,1245.33,51,35.0
4,non-sponsored,Open box,11403.29,568398,0,71037.55,26322.64,52995,12500,7462.38,...,3.00,5,1,1.63,433.20,2500,0,872.81,66,15.0
5,sponsored,Open box,11855.43,64483,5,22100.46,24327.00,33500,16995,5080.21,...,62.00,119,5,57.00,0.00,0,0,0.00,7,4.0
6,non-sponsored,Seller refurbished,172606.03,2266014,0,546610.59,23273.97,36995,12500,5281.35,...,166.67,1316,1,407.34,509.78,2999,0,982.54,32,19.0
7,sponsored,Seller refurbished,15099.50,41457,0,16980.61,22999.50,25300,19999,2132.37,...,872.50,1741,4,868.50,0.00,0,0,0.00,4,3.0
8,non-sponsored,Used,52147.60,224464,0,75243.57,22293.51,9800,10000,5392.78,...,5.00,17,1,5.58,409.07,4660,0,816.89,554,269.0
9,sponsored,Used,20125.79,258972,0,46729.88,28278.28,55351,16000,7283.85,...,356.00,1690,3,667.28,1537.19,7000,0,2488.74,68,37.0


### Observation

In [5]:
#Query to execute statistics after grouping condition and sponsored

mycursor.execute("""SELECT 

sponsored,
round(avg(seller_score),2),max(seller_score),min(seller_score),round(stddev(seller_score),2),
round(avg(item_price),2),max(item_price),min(item_price),round(stddev(item_price),2),
round(avg(best_offer_available),2),max(best_offer_available),min(best_offer_available),round(stddev(best_offer_available),2),
round(avg(items_sold),2),max(items_sold),min(items_sold),round(stddev(items_sold),2),
round(avg(shipping_price),2),max(shipping_price),min(shipping_price),round(stddev(shipping_price),2),
count(seller_name),round(avg(returns_allowed),2)

FROM ebay_items

group by sponsored
order by sponsored""")

myresult = mycursor.fetchall()
Data_without_Condition= pd.DataFrame(myresult, columns = [
    "Sponsored",
    "Average Seller Score","Max Seller score","Min Seller score","Std dev Seller score",
    "Average Item price","Max Item price","Min Item price","Std dev Item price",
    "Average Best offer","Max Best offer","Min Best offer","Std dev Best offer",
    "Average Items sold","Max Items sold","Min Items sold","Std dev Items sold",
    "Average shipping price","Max shipping price","Min shipping price","Std dev shipping price",
    "Number of seller","avg Returns allowed"
]) 
Data_without_Condition
mydb.commit()

In [29]:
Data_without_Condition.T

,0,1
Sponsored,non-sponsored,sponsored
Average Seller Score,41924.28,16905.18
Max Seller score,2266014,568398
Min Seller score,0,0
Std dev Seller score,141338,59717.5
Average Item price,25775.7,29942.1
Max Item price,9800,59999
Min Item price,10000,16000
Std dev Item price,8005.58,8782.31
Average Best offer,None,None


COMMENT : 
The average sponsored items tends to more expensive, with more shipping price and price overall. The average returns allowed is higher for sponsored. The average items sold is almost double which shows that the click through rate for sponsored ads is good. 

Shipping price, item price, items sold as indicators for sponsored.